In [ ]:
import mindspore
import mindspore.nn as nn
from mindspore import Parameter, Tensor
import numpy as np
import mindspore.ops as ops

In [ ]:
class TextCNN(nn.Cell):
    def __init__(self, embedding_size, sequence_length, num_classes, filter_sizes, num_filters, vocab_size):
        super(TextCNN, self).__init__()
        self.num_filters_total = num_filters * len(filter_sizes)
        self.filter_sizes = filter_sizes
        self.sequence_length = sequence_length
        self.W = nn.Embedding(vocab_size, embedding_size)
        self.Weight = nn.Dense(self.num_filters_total, num_classes, has_bias=False)
        self.Bias = Parameter(Tensor(np.ones(num_classes), mindspore.float32))
        self.filter_list = nn.CellList()
        for size in filter_sizes:
            seq_cell = nn.SequentialCell([
                nn.Conv2d(1, num_filters, (size, embedding_size), pad_mode='valid'),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(sequence_length - size + 1, 1))
            ])
            self.filter_list.append(seq_cell)
        
        self.expand_dims = ops.ExpandDims()
        self.transpose = ops.Transpose()
        self.concat = ops.Concat(axis=len(filter_sizes))
        self.reshape = ops.Reshape()

    def construct(self, X):
        embedded_chars = self.W(X)
        embedded_chars = self.expand_dims(embedded_chars, 1)
        pooled_outputs= []
        for conv in self.filter_list:
            pooled = conv(embedded_chars)
            pooled = self.transpose(pooled,(0,3,2,1))
            pooled_outputs.append(pooled)
            
        h_pool = self.concat((pooled_outputs[0], pooled_outputs[1], pooled_outputs[2]))
        h_pool_flat = ops.reshape(h_pool,(-1, self.num_filters_total))
        model = self.Weight(h_pool_flat) + self.Bias
        return model

In [ ]:

embedding_size = 2
sequence_length = 3
num_classes = 2
filter_sizes = [2, 2, 2]
num_filters = 3

sentences = ["i love you", "he loves me", "she likes baseball", " i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

model = TextCNN(embedding_size, sequence_length, num_classes, filter_sizes, num_filters, vocab_size)


In [ ]:
criterion = nn.SoftmaxCrossEntropyWithLogits(sparse=True,reduction='mean')
optimizer = nn.Adam(model.trainable_params(), learning_rate=0.001)

In [ ]:
inputs = Tensor([np.asarray([word_dict[n] for n in sen.split()]) for sen in sentences], mindspore.int32)
targets = Tensor([out for out in labels], mindspore.int32) 

In [ ]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

net_with_criterion = nn.WithLossCell(model, criterion)
train_network = nn.TrainOneStepCell(net_with_criterion, optimizer)
train_network.set_train()

In [ ]:
epoch = 5000
for step in range(epoch):
    loss = train_network(inputs, targets)
    
    if (step + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (step + 1), 'cost =', '{:.6f}'.format(loss.asnumpy()))

In [ ]:
test_text = 'sorry hate you'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = Tensor(tests, mindspore.int32 )

# Predict
predict = model(test_batch).asnumpy().argmax(1)
if predict[0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")